In [9]:
#conda install "gymnasium[accept-rom-license, atari]"

In [10]:
import gymnasium as gym

In [11]:
#Task 0: Recap DQN

#0.1 Provide Pseudocode for DQN


"""
target = r + gamma * maxa' [Q(s',a')]   #problem in terminal state t = if terminated: reward
DQN ← 
ERP ← {}
    t ← [s,a,r,s']
    ERP += t
        batch[s,a,r,s'] from ERP
        DQN: regress
            min MSE [Qdqn (s,a) - target]
    DQNtarget ← delayedDQN
"""

#0.2 What are the two main issues DQN faces?

"""
-moving targets (optimizing to old targets)
-overestimation bias

"""

'\n-moving targets (optimizing to old targets)\n-overestimation bias\n\n'

In [13]:
#Task 1: Basic usage

#1.1 Create environment
env = gym.make('ALE/Breakout-v5')
observation = env.reset() #

#1.2 How to create the action-state-transition loop?
ERP  = []

for _ in range(1000):

    action = env.action_space.sample()  # agent policy that uses the observation and info
    next_observation, reward, terminated, truncated, info = env.step(action) #terminated = is goal/trap state?
                                                                        #truncated = max step we allow size reached?
                                                                        #info = only info for human, not agent
    
    ERP.append((observation, action, reward, next_observation))


#1.3 How can we determine a terminal state?
if terminated or truncated:
        observation, info = env.reset()

#1.4 Which actions will be accepted?
print("Action Space", env.action_space, env.actionspace.shape)

#1.5 What properties can we expect from the returned observation?
print("Observation Space", env.observation_space, env.observation_space.shape)
"""
0 min value
255 max value
(210,160,3) shape of the Atari Screen 
unit8
"""

#1.6 How can we vectorize it (run multiple in parallel?)
#vector()
envs = gym.make('ALE/Breakout-v5', num_envs = 3) #specifiy how man you want and batch

#1.7 Why is vectorization important?
"""
Take 500 states at once of diff envs and batch_size =500 → faster
"""

AttributeError: 'AtariEnv' object has no attribute 'actionspace'

In [ ]:
#Task2: Create a DQN
import tensorflow as tf
import keras 
import numpy as np
import random

#1.1 Implement a simple DQN
def get_breakout_dqn():

  #input  = tf.float32(input)/255
  input = tf.keras.Input(shape =(210,160,3))
  x = tf.keras.layers.Conv2D(5,7, activation = "relu")(input) #input (210,160,3), output = action_space.shape

  for i in range(5):
    x = tf.keras.layers.Conv2D(5,7, activation = "relu")(x) 
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dense(4, activation=tf.nn.softmax)(x)

  dqn_model = tf.keras.Model(inputs = input, outputs = x , name ="dqn")

  return dqn_model
  
#1.2 Get the Q-value prediction on some dummy inputs

dqn = get_breakout_dqn()

sample_input = np.expand_dims(env.observation_space.sample(), 0)
dqn(sample_input) #output shape = [1,4]

#1.3 Write a function to sample an action for a given state, based on an epsilon-greedy policy 
def sample_trajectory(dqn, state, epsilon=0.2):
  terminated = False
  trajectory  = []

  state, _ = env.reset()

  while not terminated:
      
    q_vals = dqn(tf.expand_dims(state,0)).numpy()
    print(q_vals)

    #epsilon greedy or Bellman curve with softmax
    if random.random() < epsilon:
      action = env.action_space.sample()
    else: action =np.argmax(q_vals)  #remove bacth dim

    next_state, reward, terminated, _ , _ = env.step(action)
    trajectory.append((state, action, reward, next_state, terminated))

    state = next_state

  return trajectory

dqn = get_breakout_dqn()
sample_trajectory(dqn)

In [ ]:
#Task3 Updating the network

#3.1 Compute the Q-targets based on a given rewards r [batch_size], discount factor gamma and the predictions of Q(s',a) [batch_size, actions]

gamma = 0.9

q_targets  = reward + gamma * dqn(tf.expand_dims(next_state,0)).numpy() #batch dim removen?

#3.2 Expand the task above based on a binary variable t, which encodes whether the state s' is a terminal state (1. if terminal, else 0.)
if terminal:
    q_targets  = reward

#3.2 Given a (s, a, r, s', t) tuple, the target network Q_target, and the DQN Q_dqn (and the optimizer dqn_optimizer), write a function implementing a Gradient Descent Update on Q_dqn

def gradient_descent_update(sample, targt_net, dqn, gamma =0.9):

    optimizer = tf.optimizers.Adam()

    state, action, reward, next_state, terminated = sample    

    with tf.GradientTape() as tape: 
        q_values = dqn(tf.expand_dims(next_state,0))
        #q_values = tf.reduce_sum(q_values  *tf.)
        q_targets = reward +  gamma * targt_net(state, action) 

        loss = tf.reduce_mean(tf.square(q_targets - q_values))

    gradients = tape.gradient(loss, dqn.trainable_variables)
    optimizer.apply_gradients(zip(gradients, dqn.trainable_variables))

    return loss


In [ ]:
#Task4 Polyak Averaging for the target network

#4.1 Implement a function with target network Q_target, the DQN Q_dqn, and a polyak-averaging factor tau, which computes and applies a polyak-averaging step on Q_target

def apply_polyak_avg (target_net, dqn, tau):
    

In [ ]:
#Task5 Implementing the Experience Replay Buffer via TF datasets

#5.1 Write a function, which samples batch_size many trajectories (list of [s,a,r,s',t] tuples) from a verctorized Breakout game

#5.2 Pass each trajectory into a tensorflow dataset

#5.3 Provide a function, which implements appropriate preprocessing on the dataset

#5.4 Provide a function, given a list of trajectory-datasets samples steps from them (i.e. each step from any trajectory should be equally likely to be drawn!)

#5.5 Provide a function, which prepates such a combined-trajectory dataset (which constitutes an ERP!) for usage with DL

In [ ]:
#Task6 (HW): Combine the above into a full DQN implementation